In [0]:
spark.conf.set("fs.s3a.access.key", "****************")
spark.conf.set("fs.s3a.secret.key", "**********************")
spark.conf.set("fs.s3a.endpoint", "s3.amazonaws.com")

In [0]:
%sql
USE CATALOG hive_metastore;
USE traffic_db;

In [0]:
silver_df = spark.table("traffic_db.silver_traffic")


In [0]:
from pyspark.sql.functions import avg, count, when, window,col

gold_df = (
    silver_df
    .groupBy(
        "timestamp",
        "road_segment_id"
    )
    .agg(
        avg("avg_speed_kmph").alias("avg_speed"),
        avg("density_veh_per_km").alias("avg_density"),
        avg("queue_length_veh").alias("avg_queue_length"),
        count(when(col("incident_num") == 1, True)).alias("incident_count")
    )
    .withColumn(
        "congestion_level",
        when(col("avg_speed") < 20, "HIGH")
        .when(col("avg_speed") < 40, "MEDIUM")
        .otherwise("LOW")
    )
)


In [0]:
gold_df.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true")\
    .save("s3a://traffic-data-monitoring-project/delta/gold_traffic")


In [0]:
%sql
USE traffic_db;

CREATE TABLE IF NOT EXISTS gold_traffic
USING DELTA
LOCATION 's3a://traffic-data-monitoring-project/delta/gold_traffic';


In [0]:
%sql
SELECT *
FROM gold_traffic
ORDER BY window.start DESC;


window,road_segment_id,avg_speed,avg_density,avg_queue_length,incident_count,congestion_level,timestamp
null,S092,27.0,62.82,25.03,0,MEDIUM,2025-09-29T21:01:31
null,S013,46.22,26.21,8.8,0,LOW,2025-09-29T06:41:52
null,S433,6.01,95.68,54.62,0,HIGH,2025-09-30T02:40:32
null,S369,25.27,60.14,27.55,0,MEDIUM,2025-09-29T23:19:28
null,S406,47.0,30.53,8.55,0,LOW,2025-09-29T08:11:45
null,S069,5.51,86.17,61.54,0,HIGH,2025-09-30T06:36:08
null,S396,70.15,12.67,1.09,0,LOW,2025-09-28T11:54:55
null,S037,71.56,8.54,1.32,0,LOW,2025-09-28T10:25:36
null,S383,70.66,10.42,1.58,0,LOW,2025-09-28T16:04:42
null,S122,39.34,28.85,11.31,0,MEDIUM,2025-09-29T05:45:21


In [0]:
from pyspark.sql.functions import avg

gold_speed_by_road = (
    spark.table("traffic_db.gold_traffic")
         .groupBy("road_segment_id")
         .agg(avg("avg_speed").alias("avg_speed"))
         
)


In [0]:
display(gold_speed_by_road)

# Which road segments are slow / fast
# Helps traffic control prioritize roads
# This chart helps identify chronically congested road segments by comparing average vehicle speeds across locations.


road_segment_id,avg_speed
S320,24.095
S338,2.21
S105,5.29
S431,43.68
S263,44.47
S334,23.91
S028,24.97
S155,45.905
S380,3.99
S301,25.29


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import window

gold_speed_trend = (
    spark.table("traffic_db.gold_traffic")
         .groupBy('timestamp')
         .agg(avg("avg_speed").alias("avg_speed"))
         .orderBy("timestamp")
)

display(gold_speed_trend)

# Shows traffic flow trend
# Relationship between vehicle density and speed
# As density ↑ → speed ↓ (expected traffic behavior)


timestamp,avg_speed
2025-09-28T00:07:41,70.71
2025-09-28T00:12:58,70.79
2025-09-28T00:32:53,71.74
2025-09-28T00:36:25,74.88
2025-09-28T00:44:17,72.48
2025-09-28T01:01:12,68.48
2025-09-28T01:34:36,70.79
2025-09-28T01:59:04,70.24
2025-09-28T02:04:43,68.28
2025-09-28T02:04:55,70.37


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import when, count

congestion_dist = (
    spark.table("traffic_db.gold_traffic")
         .withColumn(
             "congestion_level",
             when(col("avg_speed") < 20, "High")
             .when(col("avg_speed") < 40, "Medium")
             .otherwise("Low")
         )
         .groupBy("congestion_level")
         .agg(count("*").alias("records"))
)

display(congestion_dist)

# Shows: % of traffic in each congestion level


congestion_level,records
High,39
Low,117
Medium,53


Databricks visualization. Run in Databricks to view.

In [0]:
density_speed = (
    spark.table("traffic_db.gold_traffic")
         .select("avg_density", "avg_speed")
)

display(density_speed)

# Shows inverse relationship 


avg_density,avg_speed
62.82,27.0
26.21,46.22
95.68,6.01
60.14,25.27
30.53,47.0
86.17,5.51
12.67,70.15
8.54,71.56
10.42,70.66
28.85,39.34


Databricks visualization. Run in Databricks to view.